# MS 141 Lecture 9 

# Eigenvalues and Eigenvectors

### Read: Chapter 6 Newman. Optional but recommended: Chapter 4 of Heath's book.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la

%matplotlib inline

# Set common figure parameters
newparams = {'figure.figsize': (10, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 20}
plt.rcParams.update(newparams)

$$
\newcommand{\bv}{{\mathbf{v}}}
\newcommand{\bx}{{\mathbf{x}}}
\newcommand{\ll}{{\lambda}}
$$

[Eigenvalues and eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors) are a central concept in linear algebra and matrix theory. They find application in nearly all branches of science, including circuit analysis, stability and vibration analysis, finding the energy levels and wave functions in quantum mechanics, and solving partial differential equations. They are also used in facial recognition, machine learning, stochastic methods and Markov processes, and network theory, among other applications. 

The first numerical algorithm for computing eigenvalues and eigenvectors appeared in 1929, when Richard von Mises published the *power method*. One of the most popular methods today, the *QR algorithm*, was proposed independently by John Francis and Vera Kublanovskaya in 1961. They form the foundation for modern eigenvalue methods.

## 1. Eigenvalues and eigenvectors 

Let us consider a matrix $A$ of size $N \times N$. An eigenvector $\bv$ is stretched by a factor $\ll$ equal to its eigenvalue upon applying the linear transformation represented by the matrix $A$:

$$ A \bv = \ll \bv$$

Therefore, finding the eigenvalues and eigenvectors of $A$ is equivalent to solving the homogeneous linear system $(A - \lambda I)\bv = 0$, which has solutions only if $A - \ll I$ is singular, namely if:

$$ \det(A-\ll I) = 0. $$

This equation defines a characteristic polynomial $p(\ll)$ of order $N$, with solutions equal to the eigenvalues of $A$. However, the textbook approach of finding the eigenvalues and associated eigenvectors by solving the characteristic polynomial is not suitable for computation and is almost never employed. Polynomials of order greater than 4 do not have a closed form solution, and finding the roots of an $N$-dimensional polynomial (for example, with $N$ large) is computationally costly and can lead to large errors. In particular, the polynomial roots are very sensitive to small changes in the matrix $A$, so the approach is ill-conditioned.

The characteristic polynomial always has $N$ roots, and thus a matrix $A$ of size $N\times N$ always has $N$ eigenvalues, although some may not be distinct (an eigenvalue may have multiplicity greater than 1). Most matrices of interest in science and engineering are symmetric or Hermitian and have $N$ linearly independent eigenvectors which can be made orthonormal. Some general matrices, however, are defective, and the number of eigenvectors is less than $N$. This typically occurs if the number of eigenvectors associated with one or more eigenvalues is less than their algebraic multiplicity.

Most numerical methods for computing eigenvalues and eigenvectors are based on reducing the original matrix to a simpler form whose eigenvalues and eigenvectors are easily determined. Transformations that leave eigenvalues and/or eigenvectors unchanged or easily recoverable are central to these methods. Examples of these transformations include:

- **Shift**: Subtracting a constant $\sigma$ from the diagonal elements of a matrix, using $A-\sigma I$, shifts all the eigenvalues by the constant value $\sigma$ but leaves the eigenvectors unchanged: $(A-\sigma I)\bv = (\lambda - \sigma)\bv$
- **Powers**: If $A \bv = \ll \bv$, then $A^2 \bv = \ll^2 \bv$, and more generally $A^k \bv = \lambda^k \bv$. 
- **Inverse**: The eigenvalues of $A^{-1}$ are the inverse of the eigenvalues of $A$. Multiplying from the left $A\bv=\ll\bv$ and rearranging, we get $A^{-1}\bv=\ll^{-1}\bv$.
- **Similarity**: Matrix $B$ is similar to matrix $A$ if there is a nonsingular matrix $Q$ such that 

$$ B = Q^{-1} A Q $$

The similarity transformation of a matrix $A$ leaves its eigenvalues unchanged but changes its eigenvectors, which are however recoverable:

$$ B\mathbf{u} = \ll \mathbf{u} \implies (Q^{-1}A Q) \mathbf{u} = \ll \mathbf{u} \implies A (Q\mathbf{u}) = \ll (Q\mathbf{u})$$

and thus using the transformation matrix $Q$ we can obtain the eigenvectors of $A$ as $\bv = Q \mathbf{u}$. Real symmetric matrices are a special case in which an orthogonal matrix, $Q^T=Q^{-1}$, can always be found that transforms $A$ into a similar diagonal matrix. The same is true for Hermitian matrices, for which a similarity transformation to diagonal form is accomplished by a unitary matrix $Q^\dagger = Q^{-1}$. For general matrices the situation is more complex. Only general matrices with distinct eigenvalues can be transformed to diagonal form. General matrices with eigenvalues with multipliticity greater than one are more difficult as they can at best be put into block-triangular or nearly triangular or nearly diagonal form. We will not examine these cases in detail.

**Diagonal and triangular forms** are desirable targets for similarity transformations. The eigenvalues of a diagonal matrix are its diagonal entries, and the eigenvectors are the corresponding columns of the identity matrix $I$. The eigenvalues of triangular matrices are also the diagonal entries. 

## 2. Numerical methods for eigenvalues and eigenvectors

### 2.1 Power iteration method

If the matrix $A$ is applied repeatedly to an initial guess vector $\bx_0$, the result will converge to the eigenvector with largest absolute eigenvalue.<br> 
Let us write the initial guess vector in terms of the basis of eigenvectors:

$$\bx_0= \sum_{i=1}^N \alpha_i \bv_i.$$ 

Applying the matrix $A$ repeatedly (and normalizing), we form the sequence of vectors $\bx_k = A^k \bx_0$, such that 

$$ \bx_k = \ll_1^k \left[ \alpha_1 \bv_1 + \left(
        \frac{\ll_2}{\ll_1} \right)^k \alpha_2 \bv_2 + \dots + \left(
        \frac{\ll_N}{\ll_1} \right)^k \alpha_N \bv_n \right].$$

The procedure picks out the eigenvector with largest absolute eigenvalue $\ll_1$, with a convergence rate proportional to $|\ll_2/\ll_1|$, where $\ll_2$ is the second largest eigenvalue. All other eigenvectors will be suppressed by larger factors $| \ll_j / \ll_1 | < 1$.

We compute the eigenvalue $\ll_k$ at step $k$ by computing the Ritz quotient 

$$ \ll_k = \frac{\bx_k^T A \bx_k}{\bx_k^T \bx_k}$$ 

and obtain increasingly accurate approximations of the largest eigenvalue. The power method allows us to find a *single eigenvalue and eigenvector pair* associated with the largest eigenvalue. It assumes that a largest magnitude eigenvalue exists; it fails when the largest magnitude eigenvalue has multiplicity greater than one or is a are a pair of complex conjugate numbers.

In [2]:
def power_method(A):
    
    N = np.shape(A)[0]
    nstep = 50 
    eps = 1.e-10
    
    x = np.zeros((N, nstep))
    eigmax = np.zeros(nstep) #max eigenvalue
    
    x[:, 0] = np.ones(N) + 1.e-7*np.random.rand(N) #x0
    
    for k in range(nstep):
        
        x[:,k] = x[:,k] / np.linalg.norm(x[:,k]) # normalize
        eigmax[k] = np.dot( x[:,k],np.dot(A,x[:,k]) ) #Ritz eigenvalue
        
        #debug:
        #print ('step ', k )
        #print ('evec: ', x[:,k])
        #print ('eig max: ', eigmax[k], '\n') 
        
        x[:,k+1] = np.dot(A, x[:,k]) # x_k+1 = A x_k
        
        if (abs(eigmax[k]-eigmax[k-1]) < eps): 
            print ('Power iteration converged in ', k, ' steps')
            eigmax[k+1:] = eigmax[k] #algorithm has converged
            break
            
    # returns eigenvalue, eigenvector and number of steps 
    return eigmax, x, k

In [3]:
A = np.array([[3., 1, 2, 3], \
              [1, 2, 1, 2], \
              [2, 1, 1, 1], \
             [3, 2, 1, 1]])

eigvals, eigvecs = la.eig(A)
print('Max eigval (SciPy): ', np.max(abs(eigvals)))
print('Eigvec (SciPy): ', eigvecs[:,0], '\n')

eigmax, eigvec, nstep = power_method(A)
print ('Max eigval: ', eigmax[nstep])
print ('Eigvec: ', eigvec[:,nstep])

Max eigval (SciPy):  7.090575367916475
Eigvec (SciPy):  [0.65819888 0.40481496 0.36756788 0.51748714] 

Power iteration converged in  8  steps
Max eigval:  7.090575367912985
Eigvec:  [0.65819841 0.40481509 0.36756795 0.51748759]


### 2.2 Inverse and shifted power methods

There are two useful variants of the power method. 

1) The **inverse power method** allows one to find the *smallest* eigenvalue by (formally) applying the power method to $A^{-1}$. As we discussed above, 

$$ A^{-1} \bv =\, (1 / \ll) \,\bv$$

Therefore, repeated application of $A^{-1}$ yields the inverse of the eigenvalue of minimum magnitude and its corresponding eigenvector. The inverse matrix however is never formed in the algorithm. Rather, at each step one solves the linear system:

$$ A \bx_{k+1} = \bx_k, $$
 
ideally with $LU$ decomposition to speed up the calculations. The example code given below illustrates the inverse power approach. An application of the inverse power method is finding the ground state or lowest-energy mode of a system. Coupled to the power method, one can find the range of eigenvalues in a system. In a physical problem, this may correspond to the range of time, length or energy scales involved, and thus is very useful to estimate.

2) The **shifted power method** is based on the idea that the eigenvalues of $A-\sigma I$ are shifted (by the constant value $\sigma$) compared to the eigenvalues of $A$, but the eigenvectors are unchanged: $(A-\sigma I)\bv = (\lambda - \sigma)\bv$. Of course, the shift $\sigma$ must be added to the result to obtain the eigenvalue of the original matrix $A$. One benefit of applying a shift is to accelerate convergence: since $\left| \frac{\ll_2 - \sigma}{\ll_1-\sigma} \right|$ might be larger than $\left| \ll_2/\ll_1\right|$, the convergence of the power method can be accelerated. Shfits are widely used, for example in the $QR$ method (see below), to speed up convergence.<br> 

The shift can also be applied in the conjunction with the inverse power method to obtain a specific eigenvalue. In particular, the eigenvalue of $A-\sigma I$ of smallest magnitude is $\ll-\sigma$, where $\ll$ is the eigenvalue of $A$ closest to $\sigma$. Thus, with an appropriate choice of shift, inverse iteration can be used to compute any eigenvalue and eigenvector pair of $A$, not just the extreme values.

In [4]:
def inverse_power_method(A):
    
    N = np.shape(A)[0]
    nstep = 50 
    eps = 1.e-10
    
    x = np.zeros((N, nstep))
    eigmin = np.zeros(nstep) #max eigenvalue
    
    x[:, 0] = np.ones(N) + 1.e-7*np.random.rand(N) #x0
    
    for k in range(nstep):
        
        x[:,k] = x[:,k] / np.linalg.norm(x[:,k])
        eigmin[k] = np.dot( x[:,k],np.dot(A,x[:,k]) )
        
        # A x_k+1 = x_k ; this line is different from power_method
        x[:,k+1] = np.linalg.solve(A, x[:,k])
        
        if (abs(eigmin[k]-eigmin[k-1]) < eps): 
            print ('Inverse Power iteration converged in ', k, ' steps')
            eigmin[k+1:] = eigmin[k] #algorithm has converged
            break
            
    # returns eigenvalue, eigenvector for each step 
    return eigmin, x, k

In [5]:
A = np.array([[3., 1, 2, 3], \
              [1, 2, 1, 2], \
              [2, 1, 1, 1], \
             [3, 2, 1, 1]])

# eigvecs contains the eigenvectors arranged by columns
eigvals, eigvecs = la.eig(A)

print('Min eigval (SciPy): ', np.min(abs(eigvals.real)))
print('Eigvec (SciPy): ', eigvecs.real[:,-1], '\n') #last column

eigmin, eigvec, nstep = inverse_power_method(A)
print ('Min eigval: ', eigmin[nstep])
print ('Eigvec: ', eigvec[:,nstep])

Min eigval (SciPy):  0.12094425717868382
Eigvec (SciPy):  [ 0.2268867  -0.07165632 -0.88640484  0.39708218] 

Inverse Power iteration converged in  6  steps
Min eigval:  0.12094425717879927
Eigvec:  [-0.22688689  0.07165656  0.88640481 -0.3970821 ]


### 2.3 The QR method: obtaining the full spectrum

**Similarity transformations.** 
When our goal is to compute the full set of eigenvalues of $A$, we need to go beyond the power method. 
Using a similarity transformation, we can transform the matrix $A$ into a matrix $B$ with the same eigenvalues: 

$$ B = Q^{-1}AQ,$$

If the transformed matrix $B$ is triangular or diagonal, the eigenvalues can be just read off from the diagonal matrix elements.<br>
[*Schur’s theorem*](https://en.wikipedia.org/wiki/Schur_decomposition) guarantees that for every $N\times N$ square matrix $A$ there is a similarity transformation, through a unitary matrix $Q$ (or an orthogonal matrix if $A$ is real), that transforms $A$ into a similar upper triangular matrix $B$, which has the eigenvalues of $A$ on its diagonal. A useful corollary is that if the matrix $A$ is symmetric (or Hermitian), the triangular matrix similar to $A$ is diagonal. 

**QR decomposition.** The numerical implementation of this result is based on [*$QR$ decomposition*](https://en.wikipedia.org/wiki/QR_decomposition). A general matrix $A$ can be decomposed as the product of an orthogonal matrix $Q$ (more generally, unitary) and an upper triangular matrix $R$ (also called "right-handed", hence the name $QR$):

$$ A = QR .$$

It is not difficult to understand why a matrix can be decomposed this way. Recall that the columns of an orthogonal matrix are mutually orthogonal unit vectors, so that $Q^TQ=QQ^T = I$, and thus $Q^T = Q^{-1}$. The conceptually simplest approach to carry out $QR$ decomposition is the Gram-Schmidt orthogonalization. 

Consider the [Gram-Schmidt process](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) applied to column vectors $\mathbf{a}_i$ of the matrix $A=\left[\mathbf {a}_{1}\,\,\, \mathbf{a}_2\,\,\,\ldots \,\,\,\mathbf {a}_{N}\right]$.<br> 
Define the projection onto the unit vector $\mathbf{q}$ as:

$${\rm proj}_{\mathbf {q}}\mathbf {a}= \left\langle \mathbf {q} ,\mathbf {a} \right\rangle \mathbf {q}.$$

with scalar product $\left\langle \mathbf {q} ,\mathbf {a} \right\rangle = \mathbf{q}^T \mathbf{a}$.
Then, define
$$
\begin{aligned}
\mathbf {u}_{1}&=\mathbf {a}_{1},&\mathbf {q}_{1}&={\mathbf {u}_{1} \over \|\mathbf {u}_{1}\|}\\
\mathbf {u}_{2}&=\mathbf {a}_{2}-{\rm proj}_{\mathbf {q}_{1}}\,\mathbf {a}_{2},&\mathbf {q}_{2}&={\mathbf {u}_{2} \over \|\mathbf {u}_{2}\|}\\
\mathbf {u}_{3}&=\mathbf {a} _{3}-{\rm proj}_{\mathbf {q}_{1}}\,\mathbf {a}_{3} - {\rm proj}_{\mathbf {q}_{2}}\,\mathbf {a}_{3},&\mathbf {q}_{3}&={\mathbf {u}_{3} \over \|\mathbf {u} _{3}\|}\\
&\vdots &&\vdots \\
\mathbf {u}_{N}&=\mathbf {a}_{N}-\sum _{j=1}^{N-1}\operatorname {proj}_{\mathbf {q}_{j}}\,\mathbf {a}_{N},&\mathbf{q} _{N}&={\mathbf {u}_{N} \over \|\mathbf {u}_{N}\|}
\end{aligned}
$$

We can now express the column vectors $\mathbf{a}_{i}$ using our newly computed orthonormal basis $\mathbf{q}_i$:

$$
\begin{aligned}
\mathbf {a}_{1}&=\langle \mathbf {q}_{1},\mathbf {a}_{1}\rangle\, \mathbf {q}_{1}\\
\mathbf {a}_{2}&=\langle \mathbf {q}_{1},\mathbf {a}_{2}\rangle\, \mathbf {q}_{1} + 
\langle \mathbf {q}_{2},\mathbf {a}_{2}\rangle\, \mathbf {q}_{2}\\
\mathbf {a}_{3}&=\langle \mathbf {q}_{1},\mathbf {a}_{3}\rangle\, \mathbf {q}_{1} + 
\langle \mathbf {q}_{2},\mathbf {a}_{3}\rangle\, \mathbf {q}_{2} + 
\langle \mathbf {q}_{3},\mathbf {a}_{3}\rangle\, \mathbf {q}_{3}\\
&\vdots \\
\mathbf {a}_{N}&=\sum_{j=1}^{N}\langle \mathbf {q}_{j},\mathbf {a}_{N}\rangle\, 
\mathbf {q}_{j}
\end{aligned}
$$

This can be written in matrix form as $A=QR$, where $Q$ is the matrix with column vectors $\mathbf{q}_i$:

$$ Q=\left[\mathbf {q}_{1}\,\,\, \mathbf{q}_2\,\,\, \ldots \,\,\, \mathbf {q}_{N}\right] $$ 

and $R$ is upper triangular and made up by the projections of $\mathbf{a}_i$ onto the basis vectors:

$$ R={\begin{pmatrix}\langle \mathbf {q}_{1},\mathbf {a}_{1}\rangle &\langle \mathbf {q}_{1},\mathbf {a}_{2}\rangle &\langle \mathbf {q}_{1},\mathbf {a}_{3}\rangle &\ldots \\
0&\langle \mathbf {q}_{2},\mathbf {a} _{2}\rangle &\langle \mathbf {q}_{2},\mathbf {a}_{3}\rangle &\ldots \\
0&0&\langle \mathbf {q}_{3},\mathbf {a}_{3}\rangle &\ldots \\
\vdots &\vdots &\vdots &\ddots \end{pmatrix}}. 
$$

Although the Gram-Schmidt procedure is seemingly simple, its computational cost is high due to the need to make each new basis vector orthogonal to all previous ones, and the orthogonalization quality can deteriorate due to rounding errors. In practice, $QR$ decomposition is typically carried out using *Householder reflections* or *Givens rotations*, two numerically efficient orthogonalization methods. We will not discuss these approaches here, but the interested student can read about them in Heath's book (Chapter 3). 

Note that if $A = QR$ with $Q$ orthogonal, then the matrix $B$ obtained by swapping the $Q$ and $R$ factors is similar to $A$:

$$B = R Q = \left( Q^T Q \right) R\, Q = Q^T A Q.$$

SciPy implements $QR$ decomposition as a wrapper of the Householder reflection routines in LAPACK. An example is given below.

In [6]:
A = np.array([[3., 1, 2, 3], \
              [1, 2, 1, 2], \
              [2, 1, 1, 1], \
             [3, 2, 1, 1]])

Q, R = np.linalg.qr(A)
print (Q, '\n')
print (R)

[[-6.25543242e-01  4.27160560e-01  5.79068577e-01  3.01511345e-01]
 [-2.08514414e-01 -8.81018655e-01  4.24650290e-01 -6.57865506e-17]
 [-4.17028828e-01  8.00926050e-02 -3.86045718e-02 -9.04534034e-01]
 [-6.25543242e-01 -1.86882745e-01 -6.94882293e-01  3.01511345e-01]] 

[[-4.79583152 -2.71068738 -2.50217297 -3.33623062]
 [ 0.         -1.62854963 -0.13348767 -0.58734577]
 [ 0.          0.          0.84930058  1.85301945]
 [ 0.          0.          0.          0.30151134]]


**The QR algorithm.** We are finally ready to present the $QR$ algorithm to compute eigenvalues and eigenvectors. 
The $QR$ algorithm is an iterative process that, in the limit, gives a diagonal (or in the worst case, upper
triangular) matrix similar to $A$. It involves two steps at each iteration:

1.  Factorize the matrix $A_k$ using $QR$ decomposition:

    $$A_k = Q_k R_k,$$

2.  Compute the next guess $A_{k+1}$ by swapping the $QR$ factors:

    $$A_{k+1} = R_k Q_k.$$

As discussed above, this operation gives a matrix similar to the $QR$-decomposed matrix:

$$A_{k+1}  = R_k Q_k = (Q_k^{T} Q_k) R_k Q_k = Q_k^{T} A_k Q_k. $$

Choosing $A$ as the initial matrix $A_0$, all matrices $A_k$ constructed in the iterations are similar to the original matrix $A$ and preserve its eigenvalues. As the iteration continues, the matrices $A_k$ constructed this way become increasingly closer to a diagonal matrix, while the off-diagonally elements become progressively closer to zero. To understand why, the $QR$ algorithm should be seen as a more sophisticated variation of the basic power iteration algorithm (see Heath). 

The power algorithm repeatedly multiplies $A$ times a single vector, normalizing after each iteration. The vector converges to the eigenvector associated with the largest eigenvalue. Instead, the $QR$ algorithm works with a complete basis of vectors, using $QR$ decomposition to renormalize (and orthogonalize) at each step. The approach is analogous to applying the power method to the an initial guess matrix with linearly independent and orthonormal columns, such as the identity matrix. One can show that the different columns will converge to different eigenvectors of $A$ if the matrix $Q$ is kept orthogonal. For a symmetric matrix $A$, upon convergence, 

$$AQ = QD,$$ 

where $D$ is the diagonal matrix of eigenvalues to which the $A_k$ sequence converged, 
and $Q$ is the composite of all the orthogonal similarity transformations required to get there. The columns of $Q$ are the eigenvectors of $A$. Detailed derivations are provided in Heath, chapter 4. 

A $QR$ code (no pun intended) is provided below.

In [7]:
def qr_simple(A):
    
    N = np.shape(A)[0]
    nstep = 200 
    eps = 1.e-4 # increase to 1.e-10 to 1.e-15
    
    #Anew = np.copy(A)
    evals = np.zeros((N,nstep))
    evals[:,0] = np.diag(A)
    
    err = 1.0
    
    for k in range(nstep):
        
        Q, R = np.linalg.qr(A) #A_k = QR
        A = np.dot(R,Q) #A_k+1 = RQ
        
        #norm of strict upper triangular part of A
        err = np.linalg.norm(np.tril(A, -1))

        #debug: print (err)
        evals[:,k] = np.diag(A)
        
        if (err < eps): 
            print ('QR iteration converged in ', k, ' steps')
            break
    
    #print ('QR iteration results after ', nstep, ' steps')
    # returns eigenvalue, eigenvector for each step 
    return np.diag(A), evals

In [8]:
A = np.array([[3., 1, 2, 3], \
              [1, 2, 1, 2], \
              [2, 1, 1, 1], \
             [3, 2, 1, 1]])

eigvals, eigvecs = np.linalg.eig(A)
print('SciPy eigenvalues: ', eigvals, '\n')

qr_eigvals, qr_eigvecs = qr_simple(A)
print("Our QR routine eigenvalues: ", qr_eigvals)

SciPy eigenvalues:  [ 7.09057537 -1.63656731  1.42504768  0.12094426] 

QR iteration converged in  104  steps
Our QR routine eigenvalues:  [ 7.09057537 -1.6365673   1.42504768  0.12094426]


**Computational cost.** Reducing a dense matrix to diagonal form with the QR algorithm is expensive. The QR decomposition performed in each step of the iteration has a cost $\mathcal{O}(N^3)$ for a general matrix, so iterating this procedure is very costly. The cost can be greatly reduced if the matrix is already nearly triangular before the QR algorithm iterations begin. This is the standard approach taken in modern QR algorithms, which are typically carried out in two steps:
- The matrix is first reduced to nearly triangular form (upper Hessenberg form, an upper triangular matrix with nonzero subdiagonal) using Householder reflections in $\mathcal{O}(N^3)$ operations. A symmetric (or Hermitian) matrix is reduced to tridiagonal form because a Hessenberg symmetric matrix is tridiagonal.
- The tridiagonal (symmetric case) or Hessenberg (general case) matrix is then iteratively reduced to diagonal form with the QR algorithm. The great advantage is that the work for the QR factorization in each iteration step is reduced to $\mathcal{O}(N^2)$ for a general matrix reduced to Hessenberg form and $\mathcal{O}(N)$ for a symmetric matrix in tridiagonal form. In addition, fewer iterations are required since the matrix is already nearly triangular or diagonal.  

Clever use of shifts is made in each step to speed up the convergence of obtaining each eigenvalue and eigenvector pair. The overall cost is dominated by the $\mathcal{O}(N^3)$ step to reduce the matrix to Hessenberg or triangular form, plus a smaller overhead for the actual QR iteration. In summary, modern QR algorithms can be quite complex, and they differ for symmetric and general matrices. It is advisable to use the algorithms implemented in Python or LAPACK to solve eigenvalue problems for *small to medium size* matrices.

### 2.4 Krylov subspace methods: Lanczos and Arnoldi iterations

When the matrix is large, the standard algorithm for preliminary reduction to Hessenberg form are prohibitively expensive, both in computational cost and storage. Therefore, the QR algorithm is no longer the method of choice, and obtaining all eigenvalue / eigenvector pairs may become prohibitive.  

For large matrices, and when only a small number of eigenvalue / eigenvector pairs are needed, Krylov subspace methods are the best choice, particularly for sparse matrices. In Krylov methods, one builds a smaller subspace (known as Krylov subspace) of size $M << N$ using efficient matrix-vector products. The $M\times M$ matrix in this subspace is much smaller than the original $N\times N$ matrix but its eigenvalues are equal to the lowest $M$ eigenvalues of the original matrix.

The smaller matrix is then diagonalized with the standard QR algorithm, but due to the reduced size of the subspace, the problem is manageable as it costs $\mathcal{O}(M^3)$ instead of $\mathcal{O}(N^3)$. We will outline the main ideas and refer the interested student to Heath chapter 4 for details.

For a matrix $A$ of size $N \times N$ and an arbitrary nonzero guess vector $\bx_0$, the Krylov sequence is defined as $\bx_0$, $\bx_1 = A\bx_0$, $\bx_2 = A^2\bx_0$, $\ldots$, $\bx_{k-1} = A^{k-1}\bx_0$. The elements are connected by the recursion $\bx_j = A \bx_{j-1}$. Arranging these vectors by columns in a matrix, one obtains the Krylov matrix 

$$ K_k=\left[\bx_0\quad A\bx_0\quad \ldots \quad A^{k-1}\bx_{0} \right] $$ 

and its associated Krylov subspace $\mathcal{K}_k = {\rm span}(K_k)$, of dimension $k$ if all the vectors are linearly independent. As we know from the power method, the vectors in the Krylov sequence converge to the eigenvector corresponding to the largest eigenvalue of $A$. The first few vectors are linearly independent and they span a subspace equal to their number, until they become progressively less linearly independent. We can remedy this situation with QR decomposition,

$$ K_k = Q_k R_k, $$

whereby the columns of the $k\times k$ matrix $Q_k$ form an orthonormal basis for the Krylov subspace $\mathcal{K}_k$.

Equipped with the matrix $Q_k$, we can now project the matrix $A$ onto the Krylov subspace, forming the $k \times k$ matrix $T_k=Q_k^T A Q_k$, obtained using only the upper $k \times k$ block of $A$. Focusing on the symmetric or Hermitian case, one can show that the projected matrix $T_k=Q_k^T A Q_k$ is tridiagonal (Hessenberg in the general case) and that its eigenvalues and eigenvectors correspond to the $k$ extremum eigenvalues and eigenvectors of the original matrix $A$.<br> 

Key to the algorithm is that the QR decomposition is not needed as the column vectors $\mathbf{q}_j$ of the matrix $Q_k$ can be obtained *recursively* starting from an initial guess $\mathbf{q}_0$ through efficient matrix-vector multiplications. For symmetric or Hermitian matrices, this approach for computing the extremum eigenvalues of a large and sparse matrix is called Lanczos algorithm and it computes $\mathbf{q}_j$ as a three-term recursion. For general matrices, the approach is called Arnoldi algorithm and its recursion involves more terms. 

## 3. Software for eigenvalue problems.

The choice of a method to compute eigenvalues and eigenvectors depends on how many of them are desired and the type of matrix at hand:
- For computing **all the eigenvalues (and optionally, eigenvectors)**, the standard approach is preliminary reduction to Hessenberg form, followed by the *QR algorithm*. For a symmetric or Hermitian matrix, more options are available, but almost all of them involve preliminary reduction to tridiagonal form.
- When **only one or a few eigenvalues or eigenvectors are needed**, a different set of methods are available. For small symmetric matrices, tridiagonal reduction followed by bisection for the eigenvalues and inverse power for the eigenvectors is the standard approach. 
- For **large or very large sparse matrices,** Krylov subspace methods are a widely used choice to find only a small fraction of the eigenvalues and eigenvectors. The two key methods are Arnoldi iteration for general matrices and Lanczos iteration for symmetric or Hermitian matrices.<br> 


- Wiki has a [list of eigenvalue and eigenvector algorithms](https://en.wikipedia.org/wiki/Eigenvalue_algorithm)
- [LAPACK](http://www.netlib.org/lapack/) provides a variety of [routines](https://www.netlib.org/lapack/lug/node70.html) to compute eigenvalues and eigenvectors, which are centered on the QR method.<br> Python's SciPy can access these routines through its wrappers. For larger matrices, one needs to use ScalaPack (the parallel version of LAPACK),<br> but obtaining all eigenvalues and eigenvectors could be prohibitively expensive.  
- [ARPACK](https://www.caam.rice.edu/software/ARPACK/index.html) Implements the Lanczos and Arnoldi methods to solve large eigenvalue problems. 
- Matlab and Mathematica have their own implementations for eigenvalue problems, which are typically suitable for small to medium size matrices.

## References 
- L. Trefethen, Numerical Linear Algebra
- G.H. Golub and C.F. van Loan, Matrix Computations
- Y. Saad, Iterative Methods for Sparse Linear Systems (free online download)